### Name:
> Castillo, Marvien Angel C. <br>
> Herrera, Mikhaela Gabrielle B. <br>
> Regindin, Sean Adrien I. <br>

# Setup Environment

In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# Check if CUDA is present

In [2]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov  6 03:26:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   28C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Variant 1 - C Program

In [4]:
%%writefile C_var1.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
// ***C function version
void kernel_C(float A[], float B[], float C[], size_t n, int idx[]) {
	for (int i = 0; i < n; i++) {
		if (A[i] >= B[i]) {
			C[i] = A[i];
			idx[i] = 0;
		}
		else {
			C[i] = B[i];
			idx[i] = 1;
		}
	}
}

int main(int argc, char** argv)
{
   const size_t ARRAY_SIZE = 1<<24;
   const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
   const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
//number of times the program is to be executed
   const size_t loope = 10;
//declare array
   float *C,*A,*B;
   int *idx,a;
   A = (float*)malloc(FLOAT_ARRAY_BYTES);
   B = (float*)malloc(FLOAT_ARRAY_BYTES);
   C = (float*)malloc(FLOAT_ARRAY_BYTES);
   idx = (int*)malloc(INT_ARRAY_BYTES);
   a=2;
//timer variables
  clock_t start, end;
// ***--- initialize your array here ---------
   int i;
	for (i = 0; i < ARRAY_SIZE; i++) {
		A[i] = sin(i * 0.0005) * 100.0 + 50.0;
		B[i] = cos(i * 0.0003) * 100.0 + 50.0;
	}
// fill-in cache
    kernel_C(A,B,C,ARRAY_SIZE,idx);
//time here
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loope; i++){
    start = clock();
      kernel_C(A,B,C,ARRAY_SIZE,idx );
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);

// error checking routine here --
   size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
   printf("Error count (C program): %lu\n", err_count);
  // Free memory
  free(A);
  free(B);
  free(C);
  free(idx);
  return 0;
}

Writing C_var1.c


In [5]:
%%bash
gcc C_var1.c -o C_var1 -lm

In [6]:
%%bash
./C_var1

Function (in C) average time for 10 loops is 259.889500 milliseconds to execute an array size 16777216 
Error count (C program): 0


# Variant 2 - Grid Stride Loop

In [7]:
%%writefile CUDA_var2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//Grid stride loop

//*** CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
// *** init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// *** setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Float kernel_C\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();
//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}


Writing CUDA_var2.cu


In [8]:
%%bash
nvcc CUDA_var2.cu -o CUDA_var2 -Wno-deprecated-gpu-targets

In [ ]:
%%bash
nvprof ./CUDA_var2

==1018580== NVPROF is profiling process 1018580, command: ./CUDA_var2


*** function = Float kernel_C
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==1018580== Profiling application: ./CUDA_var2
==1018580== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  122.76ms        10  12.276ms  343.45us  119.66ms  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   87.47%  1.62461s         4  406.15ms  81.704us  1.62331s  cudaMallocManaged
                    6.60%  122.51ms         1  122.51ms  122.51ms  122.51ms  cudaDeviceSynchronize
                    4.22%  78.463ms        10  7.8463ms  14.638us  78.113ms  cudaLaunchKernel
                    1.57%  29.234ms         4  7.3084ms  6.8059ms  8.1321ms  cudaFree
                    0.12%  2.1992ms       114  19.290us     127ns  1.6766ms  cuDeviceGetAttribute
                    0.01%  130.80us         1  130.80us  130.80us  130.80us  cuDeviceGetName
                    0.00%  64.217us         1  64.217us  64.217us  64.217us  cuDeviceGetPCIBusId
                    0.00%  31.000us         1  3

# Variant 3.0 - Grid Stride Loop with Prefetch

In [ ]:
%%writefile CUDA_var3.cu
// prefetch
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize(); 

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

In [ ]:
%%bash
nvcc CUDA_var3.cu -o CUDA_var3 -Wno-deprecated-gpu-targets

In [ ]:
%%bash
nvprof ./CUDA_var3

# Variant 4.0 - Grid Stride Loop with Prefetch and Page Creation

In [ ]:
%%writefile CUDA_var4.cu
//prefetch + page creation
// page creation responsible gpu fault page
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}


int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
//"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

In [ ]:
%%bash
nvcc CUDA_var4.cu -o CUDA_var4 -Wno-deprecated-gpu-targets

In [ ]:
%%bash
nvprof ./CUDA_var4

# Variant 5.0 - Grid Stride Loop with Prefetch and Page Creation + mem advise

In [ ]:
%%writefile CUDA_var5.cu
//prefetch + page creation + memadvise

#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// memory advise
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }


 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   // what if numThreads>1024, numThreads<1024?
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;

}


In [ ]:
%%bash
nvcc CUDA_var5.cu -o CUDA_var5 -Wno-deprecated-gpu-targets

In [ ]:
%%bash
nvprof ./CUDA_var5

# Variant 6.0 - CUDA classic MEMCPY

In [ ]:
%%writefile CUDA_var6.cu
//grid stride loop + memcpy
// BFF PALANG TOH SO FAR + onting research sa websites HJSHAS
// UNFINISHED

#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;

//declare array for host allocations
  float *h_A,*h_B,*h_C; 
  int *h_idx;
  cudaMalloc(&h_A, FLOAT_ARRAY_BYTES); //we dont use cudaMallocManaged with cudaMemcpy
  cudaMalloc(&h_B, FLOAT_ARRAY_BYTES);
  cudaMalloc(&h_C, FLOAT_ARRAY_BYTES);
  cudaMalloc(&h_idx, INT_ARRAY_BYTES);

// ****init host array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// device allocations
  float *d_A,*d_B,*d_C;
  int *d_idx;
  cudaMalloc(&d_A, FLOAT_ARRAY_BYTES); //we dont use cudaMallocManaged with cudaMemcpy
  cudaMalloc(&d_B, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_C, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// Copy data to device
    cudaMemcpy(d_A, h_A, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);

// Kernel launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (ARRAY_SIZE + threadsPerBlock - 1) / threadsPerBlock;

// Copy results back to host
    cudaMemcpy(h_C, d_C, FLOAT_ARRAY_BYTES, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_idx, d_idx, INT_ARRAY_BYTES, cudaMemcpyDeviceToHost);


// Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaFree(d_idx);

// Free host memory
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_idx);

  return 0;
}

In [ ]:
%%bash
nvcc CUDA_var6.cu -o CUDA_var6 -Wno-deprecated-gpu-targets

In [ ]:
%%bash
nvprof ./CUDA_var6